# Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances

from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

!wget "https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/model_config.py" -P model_config -nc
!wget "https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/utility_func.py" -P utility_func -nc
import sys
sys.path.append("model_config")
sys.path.append("utility_func")

from utility_func import *
from model_config import *

--2022-02-11 08:07:25--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/model_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6054 (5.9K) [text/plain]
Saving to: ‘model_config/model_config.py’

model_config.py     100%[===================>]   5.91K  --.-KB/s    in 0s      

2022-02-11 08:07:25 (61.2 MB/s) - ‘model_config/model_config.py’ saved [6054/6054]

--2022-02-11 08:07:25--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/utility_func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response

In [12]:
!wget "https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/data_set.csv" -P data_set -nc
processed_data = pd.read_csv("data_set/data_set.csv",
                             sep=";")

--2022-02-11 08:29:25--  https://raw.githubusercontent.com/ibrahimerdem/application_storage/main/data_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7552701 (7.2M) [text/plain]
Saving to: ‘data_set/data_set.csv’

data_set.csv        100%[===================>]   7.20M  --.-KB/s    in 0.08s   

2022-02-11 08:29:26 (86.9 MB/s) - ‘data_set/data_set.csv’ saved [7552701/7552701]



# Time-basis

In [13]:
df_a = processed_data.copy()

model_start = "07-01-2016"
model_end = "04-01-2018"
test_start = "04-01-2018"
test_end = "07-01-2018"

df_a["t_date"] = df_a["t_date"].astype("datetime64")
df_base = df_a[(df_a["t_date"]>=model_start)&(df_a["t_date"]<model_end)]

hold_df = df_a.loc[(df_a["t_date"]>=test_start)&(df_a["t_date"]<test_end),
                   ["c_id", "item_id"]]

items = df_base["item_id"].unique()
hold_df = hold_df[hold_df["item_id"].isin(items)]
hold_c = hold_df["c_id"].unique()

test_data = df_base[df_base["c_id"].isin(hold_c)]

print("train customers:", df_base["c_id"].nunique())
print("test customers:", test_data["c_id"].nunique())

train customers: 25939
test customers: 389


# Last-event

In [8]:
df_a = processed_data.copy()

df_a["t_date"] = df_a["t_date"].astype("datetime64")

model_start = "07-01-2016"
model_end = "04-01-2018"

df_base = df_a[(df_a["t_date"]>=model_start)&(df_a["t_date"]<model_end)]

test_size = 0.025

all_c = df_base["c_id"].unique()
random_selection = np.random.rand(len(all_c)) <= 1-test_size
test_c = all_c[~random_selection]
test = df_base[df_base["c_id"].isin(test_c)]

lasts = test.groupby("c_id", as_index=False).last()
hold_table = lasts[["c_id", "item_id"]]
last_tr = lasts["t_id"].unique()
test_data = test[~(test["t_id"].isin(last_tr))]

df_base = df_base[~(df_base["t_id"].isin(last_tr))]

print("train customers:", df_base["c_id"].nunique())
print("test customers:", test_data["c_id"].nunique())

train customers: 25939
test customers: 640


In [ ]:
user = pd.pivot_table(data=df_base,
                      values="payment",
                      index="c_id",
                      columns="item_id")
user = user.apply(lambda x: x/x, axis=1)
user = user.fillna(0)

# User-based Similarity

In [ ]:
customer_list = test_data["c_id"].unique()

main_table = hold_table
main_table["item_id"] = main_table["item_id"].astype("int")
main_table["hit"] = 0
main_table["r_list"] = 0

k = 20
n = 10

hitt = 0
rect = 0
pret = 0
ndct = 0

counter = len(customer_list)
for i in range(counter):
    c = customer_list[i]
    rel = main_table.loc[main_table["c_id"]==c, "item_id"].unique()

    score = 1 - pairwise_distances(user[user.index==c], user, metric="cosine")
    nn = score[0].argsort()[::-1][1:n+1]
    rank_table = list()
    for h in nn: 
        rank_table.append(user[user.index==h+1].values)
    ranks = np.mean(rank_table, axis=0)[0].argsort()[::-1][:k]
    ret = list(user.columns[ranks])
    rlist = ",".join([str(i) for i in ret])
    main_table.loc[main_table["c_id"]==c, "r_list"] = rlist 

    bl = np.isin(ret, rel)*1
    rec = np.sum(bl)/len(rel)
    pre = average_precision(bl)
    ndc = ndcg_k(bl, len(bl))
    h = np.max(bl)
    main_table.loc[main_table["c_id"]==c, "hit"] = h
    hitt += h
    rect += rec
    pret += pre
    ndct += ndc          

hit = hitt/counter
recall = rect/counter
map = pret/counter 
ndcg = ndct/counter

result = main_table.groupby("c_id").agg(
    {"item_id": lambda x: ",".join([str(i) for i in x]),
     "hit": "max",
     "r_list": "last"})

print(f"Hit@{k} : {np.round(hit, 4)}")
print(f"Recall@{k} : {np.round(recall, 4)}")
print(f"MAP@{k} : {np.round(map, 4)}")
print(f"NDCG@{k} : {np.round(ndcg, 4)}")

print(result.hit.value_counts())
print(result.hit.value_counts()/len(result))

Hit@20 : 0.2424
Recall@20 : 0.2424
MAP@20 : 0.1772
NDCG@20 : 0.2073
0    500
1    160
Name: hit, dtype: int64
0    0.757576
1    0.242424
Name: hit, dtype: float64
